# Foodcast - food forecasting
Dans ce notebook, on se propose d'explorer et comprendre les différentes briques élémentaires d'un problème de prévision hebdomadaire de chiffre d'affaire. L'introduction de [MLflow](https://mlflow.org/) se fera dans un second temps.

Le jeu de données est [celui-ci](https://www.kaggle.com/henslersoftware/19560-indian-takeaway-orders) et se trouve dans le répertoire `data` du projet. Le répertoire `data/raw` contient les données telles quelles.

Le scénario est le suivant : une chaîne de restauration possède plusieurs restaurants dans une ville donnée. Chaque restaurant enregistre la totalité de ses ventes. La chaîne veut pouvoir prédire son volume de chiffre d'affaire d'une semaine sur l'autre, tous établissements confondus.

L'idée est de se mettre en conditions de production, avec un rythme de prédiction hebdomadaire. Chaque semaine, un pipeline de traitement et de prévision est à activer. Pour émuler ce genre d'environnements, nous avons découpé les données en batchs hebdomadaires dans `data/batchs` à l'aide du script `reformatting.py`. Par souci de simplicité, les semaines sont identifiées avec un nombre entier

<img src="images/data.png" style="width: 400px;"/>

Dans ce notebook, on aborde les étapes suivantes du pipeline :
* [Importation des librairies](#part0)
* [Chargement et nettoyage des données](#part1)
* [Feature engineering sur le jeu d'entraînement (offline)](#part2)
* [Entraînement d'un modèle prédictif](#part3)
* [Feature engineering sur le jeu de prédiction (online)](#part4)
* [Prévision et visualisation](#part5)
* [Incertitudes de modélisation](#part6)

**A noter :** toutes les fonctions élémentaires qui implémentent ces étapes sont **déjà codées**. Dans ce notebook, il s'agit simplement de se familiariser avec.

# Importation des librairies

<a class='anchor' id='part0'></a>

La structure du projet est la suivante :

<img src="images/tree.png" style="width: 300px;"/>

**A noter :** il manque le répertoire `application` du template DDD. Ce répertoire est celui qui hébergera tous nos appels à [MLflow](https://mlflow.org/), et sera construit au fur et à mesure de la présente formation.

Dans la suite, nous aurons besoin des fonctions suivantes. Nous allons illustrer leur usage dans ce notebook.

In [1]:
import sys
sys.path.append('..')
import yaml
import logging
import logging.config
import pandas as pd
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 400)
from foodcast.domain.transform import etl
from foodcast.domain.feature_engineering import features_offline, features_online
from foodcast.domain.forecast import span_future, cross_validate, plotly_predictions
from foodcast.domain.multi_model import MultiModel
from sklearn.ensemble import RandomForestRegressor
import foodcast.settings as settings
import plotly.graph_objects as go

with open(settings.LOGGING_CONFIGURATION_FILE, 'r') as f:
    logging.config.dictConfig(yaml.safe_load(f.read()))

%load_ext autoreload
%autoreload 2

/opt/anaconda3/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:155: FutureWarning: Model's `predict` method contains invalid parameters: {'X'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")


************************************************************
USING default value : foodcast.settings.dev
************************************************************


# Chargement et nettoyage des données

<a class='anchor' id='part1'></a>

Dans cette partie, on se focalise sur le pré-traitement des données.

### Fonctions élémentaires

Le pré-traitement qui suit se déroule en quatre étapes, chacune étant encodée dans une fonction élémentaire :
* `extract` : charge les données d'un restaurant dans un intervalle de temps défini par l'utilisateur
* `clean` : nettoie le jeu de donnée correspondant :
    * homogénéisation des noms de colonnes
    * corrections des dtypes
    * agrégation du montant à la maille transaction
    * suppression des colonnes inutiles
    * tri chronologique
* `merge` : fusionne les données de chaque restaurant en un seul dataframe représentant la chaîne de restauration
* `resample` : réechantillonne le jeu de données à la maille heure

### Exercices

Ces quatre fonctions sont encapsulées dans une seule et unique fonction maître, appelée `etl`, qui fait l'objet du prochain exercice.

In [ ]:
etl??

**Exercice :** extraire un jeu de données entièrement pré-traité pour les semaines 197 à 200.

**Indice :** le nom du répertoire où se situe les données est stocké dans `settings.DATA_DIR`.

In [2]:
df = etl(settings.DATA_DIR, 197, 200)
df.head(2)

2025-12-03 14:52:12 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/infrastructure/extract.py - INFO - extract: shape = (2158, 6)
2025-12-03 14:52:12 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/infrastructure/extract.py - INFO - extract: shape = (3247, 6)
2025-12-03 14:52:12 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/transform.py - INFO - clean: shape = (380, 3)
2025-12-03 14:52:12 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/transform.py - INFO - clean: shape = (565, 3)
2025-12-03 14:52:12 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/transform.py - INFO - merge: shape = (945, 2)
/Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/transform.py:79: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return df.resample('1H', on

,order_date,cash_in
0,2018-10-08 09:00:00,89.55
1,2018-10-08 10:00:00,0.00


**Exercice :** tracer le chiffre d'affaire en fonction du temps avec [plotly](https://plotly.com/python/line-charts/#line-plot-with-goscatter).

In [11]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df['order_date'],
        y=df['cash_in'],
        line_color='dodgerblue'
    )
)
fig.update_layout(
    title='Cash-in',
    xaxis_title='date',
    yaxis_title='dollars',
    font=dict(
        family='Computer Modern',
        size=18,
        color='#7f7f7f'
    )
)

# Feature engineering sur le jeu d'entraînement (offline)

<a class='anchor' id='part2'></a>

Dans cette partie, on se focalise sur le feature engineering et la création du jeu d'entraînement.

### Fonctions élémentaires
Le feature engineering qui suit se déroule en trois étapes, chacune étant encodée dans une fonction élémentaire :
* `dummy_day` : encode le jour de la semaine en 6 variables binaires.
* `hour_cos_sin` : encode l'heure de la journée en 2 variables continues.
* `lag_offline` : récupère le chiffre d'affaire d'une semaine dans le passé.

Au niveau implémentation, `lag_offline` est un simple `shift` de la cible sur le jeu d'entraînement.

### Exercices

Ces trois fonctions sont encapsulées dans une seule et unique fonction maître, appelée `features_offline`, qui fait l'objet du prochain exercice.

In [5]:
features_offline??

Signature:
features_offline(
    df: pandas.core.frame.DataFrame,
    degree: int = 1,
    lag_in_week: int = 1,
) -> pandas.core.frame.DataFrame
Source:   
@log_return_shape
def features_offline(df: pd.DataFrame, degree: int = 1, lag_in_week: int = 1) -> pd.DataFrame:
    """
    Offline feature engineering with enough history to compute lags.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe to add features on.
    degree : int, optional
        Degree of the sines and cosines computed, by default 1.
    lag_in_week : int, optional
        Number of weeks to lag, by default 1.

    Returns
    -------
    pd.DataFrame
        Input dataframe with additional features.
    """
    df = dummy_day(df)
    df = hour_cos_sin(df, degree=degree)
    df = lag_offline(df, lag_in_week=lag_in_week)
    return df
File:      ~/Documents/SIAD/M2/MLOPS/tp_mlflow/foodcast/domain/feature_engineering.py
Type:      function

**Exercice :** effectuer le feature engineering sur le jeu d'entraînement obtenu précédemment.

In [7]:
df = features_offline(df)

2025-12-03 15:02:37 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/feature_engineering.py - INFO - dummy_day: shape = (659, 8)
2025-12-03 15:02:37 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/feature_engineering.py - INFO - hour_cos_sin: shape = (659, 10)
2025-12-03 15:02:37 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/feature_engineering.py - INFO - lag_offline: shape = (491, 11)
2025-12-03 15:02:37 - /Users/juleslefevre/Documents/SIAD/M2/MLOPS/tp_mlflow/notebooks/../foodcast/domain/feature_engineering.py - INFO - features_offline: shape = (491, 11)


**Exercice :** vérifier à la main sur une ligne ou deux la validité de la variable `lag_1W` créée.

In [12]:
df.tail(1)

,order_date,cash_in,day_1,day_2,day_3,day_4,day_5,day_6,hour_cos_1,hour_sin_1,lag_1W
490,2018-11-04 19:00:00,41.4,False,False,False,False,False,True,0.258819,-0.965926,130.55


In [13]:
df[df.order_date == '2018-10-28 19:00:00']

,order_date,cash_in,day_1,day_2,day_3,day_4,day_5,day_6,hour_cos_1,hour_sin_1,lag_1W
322,2018-10-28 19:00:00,130.55,False,False,False,False,False,True,0.258819,-0.965926,162.15


### Séparation variable/cible

On sépare le jeu de données obtenu en variables explicatives et variable cible, en prenant soin de conserver l'information de la date.

In [14]:
# Uncomment to achieve exercise

x_train = df.drop(columns=['cash_in'])
y_train = df[['order_date', 'cash_in']]
x_train = x_train.set_index('order_date')
y_train = y_train.set_index('order_date')['cash_in']

# Entraînement d'un modèle prédictif

<a class='anchor' id='part3'></a>

Dans cette partie, on se focalise sur l'entraînement d'un modèle prédictif et sa validation.

### Un premier modèle
Pour commencer, on se propose d'introduire un modèle de type forêt aléatoire avec 10 arbres. 

**Exercice :** créer une instance de `RandomForestRegressor` contenant 10 arbres, avec une graine aléatoire fixée (de votre choix).

In [15]:
simple_model = RandomForestRegressor(n_estimators=10, random_state=42)

### Validation croisée temporelle sur le jeu d'entraînement

La validation croisée temporelle est naturelle pour un problème de forecast. Elle est en fait naturelle pour tout cycle de vie des modèles sujet à une dérive des données.

<img src="images/timeseriessplit.png" style="width: 300px;"/>

C'est la fonction `cross_validate` qui l'implémente, en se basant sur [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) de scikit-learn.

In [16]:
cross_validate??

Signature:
cross_validate(
    model: sklearn.base.BaseEstimator,
    x: pandas.core.frame.DataFrame,
    y: pandas.core.frame.DataFrame,
    n_fold: int = 10,
) -> Tuple[<built-in function array>, pandas.core.frame.DataFrame]
Source:   
def cross_validate(
    model: BaseEstimator,
    x: pd.DataFrame,
    y: pd.DataFrame,
    n_fold: int = 10
) -> Tuple[np.array, pd.DataFrame]:
    """
    Custom cross-validation, compatible with a sklearn TimeSeriesSplit.
    Return MAEs (Mean Absolute Errors) as well as a dataframe of predictions.

    Parameters
    ----------
    model : BaseEstimator
        Model to cross-validate.
    x : pd.DataFrame
        Input features of the training set.
    y : pd.DataFrame
        Input labels of the training set.
    n_fold : int
        Number of temporal cross-validation folds, by default 10.

    Returns
    -------
    Tuple[np.array, pd.DataFrame]
        maes: list of cross-validation MAEs (Mean Absolute Errors).
        preds: pd.DataFrame wit

**Exercice :** valider le modèle sur le jeu d'entraînement avec trois répétitions (folds).

In [ ]:
maes, preds = None, None

**Question :** en quelle unité s'exprime la MAE ? Compte tenu des patterns de consommation, est-ce vraiment un indicateur de performance pertinent ?

### Graphe des prédictions

On peut tracer les prédictions obtenues via la fonction `plotly_predictions`.

In [ ]:
plotly_predictions??

**Exercice :** tracer les prédictions obtenues par validation croisée, en regard de la vérité attendue.

In [ ]:
pass

### Entraînement sur la totalité du jeu de données

C'est la méthode `fit` de `RandomForestRegressor` qui est à l'oeuvre.

**Exercice :** entraîner le modèle sur tout le jeu d'entraînement.

**Indice :** on utilisera les dataframes `x_train` et `y_train` obtenus précédemment.

In [ ]:
pass

# Feature engineering sur le jeu de prédiction (online)

<a class='anchor' id='part4'></a>

Dans cette partie, on se focalise sur le feature engineering et la création du jeu de prédiction.

### Fonctions élémentaires
Le feature engineering qui suit se déroule en quatre étapes, chacune étant encodée dans une fonction élémentaire :
* `span_future`: génère les dates de prédiction dans le futur.
* `dummy_day` : encode le jour de la semaine en 6 variables binaires.
* `hour_cos_sin` : encode l'heure de la journée en 2 variables continues.
* `lag_online` : récupère le chiffre d'affaire d'une semaine dans le passé.

### Pourquoi lag offline et lag online ?

Par rapport au jeu d'entraînement, il est plus difficile de calculer un lag de chiffre d'affaire sur le jeu de prédiction car ce dernier est par définition dans le futur, et ne contient aucune information passée.

Deux méthodes sont possibles :

* **la méthode coûteuse en RAM :** il s'agit de concaténer `train` et `future` et de réaliser un `shift`. Si le `train` est volumineux, on occupe beaucoup d'espace mémoire alors que seule une petite quantité d'information nous intéresse.
* **la méthode recommandée :** il s'agit de charger uniquement les observations de la semaine passée, `past`, de concaténer avec `future`, et d'effectuer un `shift`. On utilise alors peu de mémoire (une semaine de données seulement).

### Exercices

Ces quatre fonctions sont encapsulées dans les fonctions `span_future` et `features_online`, qui font l'objet des prochains exercices.

Tout d'abord, nous générons un jeu `past` qui correspond à la semaine juste avant la semaine de prédiction.

**Exercice :** créer un jeu de données nettoyées `past` décrivant la semaine 200.

**Indice :** on pourra réutiliser la fonction `etl`.

In [ ]:
past = None

Ensuite, il faut générer le jeu de prédiction, c'est la fonction `span_future` qui s'y attèle.

In [ ]:
span_future??

**Exercice :** générer un dataframe de dates à prédire dans le futur du jeu d'entraînement.

**Indice :** on pourra utiliser `past['order_date'].max()` en guise de point de départ.

In [ ]:
future = None

Toutes les étapes de feature engineering online sont rassemblées dans la fonction `features_online`.

In [ ]:
features_online??

**Exercice :** créer un jeu de prédiction `future`, en utilisant la méthode recommandée de lag online (voir ci-dessus).

In [ ]:
future = None

**Exercice :** vérifier à la main sur une ligne ou deux la validité de la variable créée.

**Indice :** on pourra par exemple regarder en détail les variables calculées au 5 novembre 2018 à 18h.

In [ ]:
pass

On garde l'information sur la date dans l'index du jeu de prédiction.

In [ ]:
# Uncomment to achieve exercise

# future = future.set_index('order_date')

# Prévision et visualisation

<a class='anchor' id='part5'></a>

Dans cette partie, on se concentre sur la prédiction du chiffre d'affaire dans le futur du jeu d'entraînement.

### Prédiction du chiffre d'affaires

C'est la méthode `predict` de `RandomForestRegressor` qui entre en jeu.

**Exercice :** prédire le chiffre d'affaire sur le jeu de prédiction. 

**Indice :** on rangera les prédictions dans un `DataFrame` avec le même index que `future` et une seule colonne intitulée `y_pred_simple`.

In [ ]:
y_pred = None

### Visualisation des prédictions

On peut tracer les prédictions obtenues via la fonction `plotly_predictions`.

**Exercice :** tracer les prévisions de chiffre d'affaires sur le jeu de prédiction. 

In [ ]:
pass

# Incertitudes de modélisation

<a class='anchor' id='part6'></a>

Dans cette partie, on se propose d'ajouter une incertitude à nos prédictions. Une manière simple d'obtenir une incertitude sur les résultats est de perturber aussi bien le jeu de données que le modèle, comme illustré ci-contre.

<img src="images/multimodel.png" style="width: 500px;"/>

C'est la classe `MultiModel` qui implémente ce schéma : 
* bootstrap sur les données
* variation de graine aléatoire du modèle (si elle existe). 

D'un point de vue design pattern, il s'agit d'un [décorateur](https://refactoring.guru/design-patterns/decorator).

In [ ]:
MultiModel?

**Exercice :** implémenter un `MultiModel` contenant 10 répliques de `simple_model`.

In [ ]:
multi_model = None

### Validation croisée temporelle sur le jeu d'entraînement

**Exercice :** valider le modèle sur le jeu d'entraînement avec trois répétitions (folds).

**Indice :** la syntaxe est identique à celle utilisée pour `simple_model`.

In [ ]:
maes, preds = None, None

**Question :** quelle est la moyenne et l'écart-type des MAEs sur chaque répétition de la validation croisée ?

**Indice :** `axis=1`

In [ ]:
pass

### Graphe des prédictions

On peut tracer les prédictions obtenues via la fonction `plotly_predictions`. Cette fonction gère bien les prédictions d'un multi-modèle. En particulier, elle ne trace pas une courbe de prédiction mais une *plage* de prédictions.

**Exercice :** tracer les prédictions obtenues par validation croisée, en regard de la vérité attendue.

**Indice :** la syntaxe est identique à celle utilisée pour `simple_model`.

In [ ]:
pass

### Entraînement sur la totalité du jeu de données

C'est la méthode `fit` de `MultiModel` qui est à l'oeuvre.

In [ ]:
MultiModel.fit??

**Exercice :** entraîner le modèle sur tout le jeu d'entraînement.

**Indice :** on utilisera les dataframes `x_train` et `y_train` obtenus précédemment.

In [ ]:
pass

### Prédiction du chiffre d'affaires

C'est la méthode `predict` de `MultiModel` qui entre en jeu.

In [ ]:
MultiModel.predict??

**Exercice :** prédire le chiffre d'affaire sur le jeu de prédiction.

**Indice :** on fera attention à l'API non-standard de la méthode `predict`. En effet, la méthode `predict` contient un argument supplémentaire, le `context`. Cette spécificité est nécessaire pour être compatible avec [MLflow](https://mlflow.org/), mais deviendra invisible par la suite.

In [ ]:
y_pred = None

### Visualisation des prédictions avec incertitude

On peut tracer les prédictions obtenues via la fonction `plotly_predictions`.

**Exercice :** tracer les prévisions de chiffre d'affaires sur le jeu de prédiction. 

**Indice :** la syntaxe est identique à celle utilisée pour `simple_model`.

In [ ]:
pass

# Félicitations !

Vous maîtrisez maintenant le code food forecasting et vous avez implémenté un modèle [compatible avec MLflow](https://www.mlflow.org/docs/latest/models.html#model-customization) !

### Pour aller plus loin

Dans le notebook suivant, nous aborderons :
* le tracking et la reproductibilité
* le packaging de modèles
* la visualisation dans l'interface graphique